In [1]:
import os

directory = 'data'

texts = []
labels = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):  
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            for line in file:
                if '|' in line:
                    parts = line.rsplit('|', 1)
                    if parts[1].strip() == "":
                        continue
                    text = parts[0].strip()
                    label = parts[1].strip() if len(parts) > 1 else 'O'  
                    texts.append(text)
                    labels.append(label)

In [2]:
from collections import Counter
Counter(labels)

Counter({'O': 10209,
         'AttackRansom': 1494,
         'AttackRansom, AttackDatabreach': 24,
         'DiscoverVulnerability': 1467,
         'AttackDatabreach': 1405,
         'PatchVulnerability': 949,
         'AttackPhishing': 1283,
         'PatchVulnerability, DiscoverVulnerability': 83,
         'DiscoverVulnerability , PatchVulnerability': 40,
         'DiscoverVulnerability, PatchVulnerability': 121,
         'AttackPhishing, AttackDatabreach': 26,
         'AttackDatabreach, AttackPhishing': 21,
         'DiscoverVulnerability, AttackDatabreach': 22,
         'PatchVulnerability, AttackRansom': 4,
         'PatchVulnerability , DiscoverVulnerability': 9,
         'DiscoverVulnerability, AttackPhishing': 2,
         'AttackRansom, AttackDatabreach, AttackPhishing': 1,
         'AttackDatabreach, AttackRansom': 14,
         'PatchVulnerability , AttackDatabreach': 1,
         'AttackDatabreach, DiscoverVulnerability': 8,
         'AttackDatabreach, PatchVulnerability': 3,

In [3]:
temp  = []
for label in labels:
    for t in label.split(','):
        temp.append(t.strip())

In [4]:
Counter(temp)

Counter({'O': 10209,
         'AttackRansom': 1581,
         'AttackDatabreach': 1571,
         'DiscoverVulnerability': 1771,
         'PatchVulnerability': 1228,
         'AttackPhishing': 1374})

In [5]:
import pandas as pd


df = pd.DataFrame({
    'Text': texts,
    'Label': labels
})

all_labels = ["AttackDatabreach", "AttackPhishing", "AttackRansom", "DiscoverVulnerability", "PatchVulnerability","O"]


def label_columns(row, label_list):
    label_data = {label: 0 for label in label_list}
    entries = row['Label'].split(',')
    for entry in entries:
        entry = entry.strip()
        if entry in label_data:
            label_data[entry] = 1
    return pd.Series(label_data)

label_df = df.apply(lambda row: label_columns(row, all_labels), axis=1)


final_df = pd.concat([df['Text'], label_df], axis=1)
final_df


,Text,AttackDatabreach,AttackPhishing,AttackRansom,DiscoverVulnerability,PatchVulnerability,O
0,Attackers start wiping data from CouchDB and H...,0,0,0,0,0,1
1,Researchers are now observing similar destruct...,0,0,0,0,0,1
2,Security researchers Victor Gevers and Niall M...,0,0,0,0,0,1
3,The two have put together spreadsheets on Goog...,0,0,0,0,0,1
4,"In the case of Hadoop, a framework used for di...",0,0,0,0,0,1
...,...,...,...,...,...,...,...
17264,"""Ransomware has become a billion-dollar cash c...",0,0,0,0,0,1
17265,In order to help prevent falling victim to ran...,0,0,0,0,0,1
17266,"Launched in 2016, the No More Ransom scheme br...",0,0,1,0,0,0
17267,The portal is available in 29 languages and si...,0,0,0,0,0,1


In [6]:
labels = ["AttackDatabreach", "AttackPhishing", "AttackRansom", "DiscoverVulnerability", "PatchVulnerability", "O"]

# Create label2id dictionary
label2id = {label: i for i, label in enumerate(labels)}

# Create id2label dictionary
id2label = {i: label for i, label in enumerate(labels)}

# Output the dictionaries
print("label2id:", label2id)
print("id2label:", id2label)


label2id: {'AttackDatabreach': 0, 'AttackPhishing': 1, 'AttackRansom': 2, 'DiscoverVulnerability': 3, 'PatchVulnerability': 4, 'O': 5}
id2label: {0: 'AttackDatabreach', 1: 'AttackPhishing', 2: 'AttackRansom', 3: 'DiscoverVulnerability', 4: 'PatchVulnerability', 5: 'O'}


In [7]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
from transformers import DataCollatorWithPadding

dataset = Dataset.from_pandas(final_df)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Convert integer labels to floats for BCEWithLogitsLoss
def format_labels(example):
    labels = [float(example[col]) for col in ['AttackDatabreach', 'AttackPhishing', 'AttackRansom', 
                                              'DiscoverVulnerability', 'PatchVulnerability', 'O']]
    return {'labels': labels}

tokenized_datasets = tokenized_datasets.map(format_labels)

# Split the dataset into train, validation, and test sets
train_testvalid = tokenized_datasets.train_test_split(test_size=0.2, shuffle=True)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=True)
train_dataset = train_testvalid['train']
valid_dataset = test_valid['train']
test_dataset = test_valid['test']

# Metrics computation
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Apply sigmoid function to logits and threshold at 0.5 for binary prediction
    preds = (1 / (1 + np.exp(-logits)) > 0.5).astype(float)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {'precision': precision, 'recall': recall, 'f1': f1}



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# Training
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01
)

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6, problem_type="multi_label_classification")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/home/is1ab/anaconda3/envs/coref/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: t111599004 (ntut-biolab). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
# Evaluate on test data
trainer.evaluate(test_dataset)


In [ ]:
# Evaluate on test data
trainer.evaluate(test_dataset)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub(commit_message="Training complete")